In [ ]:
from datasets import SHD_dataloaders, SSC_dataloaders, GSC_dataloaders
from config import Config
from snn_delays import SnnDelays
import torch
from snn import SNN
import utils

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n=====> Device = {device} \n\n")

In [ ]:
config = Config()

if config.model_type == 'snn':
    model = SNN(config).to(device)
else:
    model = SnnDelays(config).to(device)

if config.model_type == 'snn_delays_lr0':
    model.round_pos()


print(f"===> Dataset    = {config.dataset}")
print(f"===> Model type = {config.model_type}")
print(f"===> Model size = {utils.count_parameters(model)}\n\n")

In [ ]:
if config.dataset == 'shd':
    train_loader, valid_loader = SHD_dataloaders(config)
    test_loader = None
elif config.dataset == 'ssc':
    train_loader, valid_loader, test_loader = SSC_dataloaders(config)
elif config.dataset == 'gsc':
    train_loader, valid_loader, test_loader = GSC_dataloaders(config)
else:
    raise Exception(f'dataset {config.dataset} not implemented')

In [ ]:
model.train_model(train_loader, valid_loader, test_loader, device)

In [ ]:
# Get one batch from the train_loader
train_iter = iter(train_loader)
first_batch = next(train_iter)

In [ ]:
# Assuming the batch is a tuple of (data, labels)
data, labels, _ = first_batch

# Print the first data point and its label
print(data[0], labels[0])

In [ ]:
data = data.permute(1,0,2).float().to(device)  #(time, batch, neurons)
output = model.forward(data)

In [ ]:
output

In [ ]:
x = torch.rand(10, 2) > 0.2
x

In [ ]:
x[:, 1]

In [ ]:
model.train_model(train_loader, valid_loader, test_loader, device)

# **Nouvelle classe linéaire**

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class DaleLinear(nn.Module):
    